Some experiments

In [ ]:
%pip install sentence_transformers torch ipywidgets

In [1]:
import torch.nn.functional as F
import json
import os
import torch
import pickle

from bs4 import BeautifulSoup


from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [45]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0) # nonmasked -> 0
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None] # average of masked, as attention_mask is 1 or 0

input_texts = ['query: how much protein should a female eat',
               'query: summit define',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."]
tokenizer = AutoTokenizer.from_pretrained('./data/original')
model = AutoModel.from_pretrained('./data/original')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

[[90.31377410888672, 69.97345733642578], [70.04136657714844, 91.45667266845703]]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./data/original')
model = AutoModel.from_pretrained('./data/original')

In [40]:

v1=torch.tensor([[1,1,1,1],[1,1,1,1]])
v2=v1*3
v1[1][2]=0
v2.masked_fill(v1.bool(),5)
# v1.bool()

tensor([[5, 5, 5, 5],
        [5, 5, 3, 5]])

In [61]:
# batch_dict['attention_mask'].bool().shape
# torch.Size([4, 75])
# batch_dict['attention_mask'][...,None].bool().shape
# torch.Size([4, 75, 1])
# v1[...,None]
# batch_dict['attention_mask'].sum(dim=1).shape
v1.sum(dim=1)

tensor([4, 3])

In [4]:
class EmbedTest:
    def __init__(self) -> None:
        self._tokenizer = AutoTokenizer.from_pretrained('./data/original')
        self._model = AutoModel.from_pretrained('./data/original')
    def _average_pool(self,last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0) # nonmasked -> 0
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None] # average of masked, as attention_mask is 1 or 0
    def generateEmbeddings(self,texts):
        batch_dict = self._tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')      
        outputs = self._model(**batch_dict)
        embeddings = self._average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        # normalize embeddings
        return F.normalize(embeddings, p=2, dim=1)

embedder = EmbedTest()


Calculate article embeddings

In [10]:

def readArticle(id):
     with open(f"./data/articles/{id}") as f:
         article = json.load(f)
         text = BeautifulSoup(article["body"]).get_text()
         [embedding] = embedder.generateEmbeddings([f"passage:{text}"])
         return (article["id"],article["name"],embedding)

artfiles = os.listdir("./data/articles/")
articles = list(map(readArticle,artfiles))

with open("./data/trained/articles.dat","wb") as f:
    pickle.dump(articles,f)


read articles and embeddings

In [6]:
with open("./data/trained/articles.dat","rb") as f:
    articles = pickle.load(f)

In [55]:
def lookup(q):
    [queryembed] = embedder.generateEmbeddings(f"query:{q}")
    def score(art):
         return (((art[2] @ queryembed) * 100).item(),art)
    scored = list(map(score, articles))
    scored.sort(reverse=True)
    def extract(art):
         return (art[0],art[1][0],art[1][1],f"https://support.basistechnologies.com/hc/en-us/articles/{art[1][0]}")
    return list(map(extract,scored[0:5]))

def test(query):
     result = lookup(query)
     for r in result:
          print(r)
     print()

# print(lookup("task"))

# print(lookup("short dump"))
# print(lookup("test impact"))

test("import errors")
test("unexpected result")
test("task")

(81.06541442871094, 13862240080285, 'ActiveControl Error: (addtobuffer has problems with data- and/or cofile)', 'https://support.basistechnologies.com/hc/en-us/articles/13862240080285')
(81.03059387207031, 14868550624541, 'ActiveControl How To: Configured to send automated e-mails notifications?', 'https://support.basistechnologies.com/hc/en-us/articles/14868550624541')
(81.021240234375, 13430714503453, 'ActiveControl How to: Stop Auto-Import Jobs from De-scheduling', 'https://support.basistechnologies.com/hc/en-us/articles/13430714503453')
(80.37882232666016, 14706112950173, 'Errors about "unprocessed linkages" and "linkages with errors" when running Test Impact Radar (0043)', 'https://support.basistechnologies.com/hc/en-us/articles/14706112950173')
(79.86550903320312, 14111832764829, 'Can we manage client 000 using ActiveControl (create and import transports)? ', 'https://support.basistechnologies.com/hc/en-us/articles/14111832764829')

(74.66816711425781, 211706683, 'Why do I not ge